In [90]:
import pandas as pd
import spacy
import en_core_web_sm
import random
from spacy import displacy

In [2]:
df = pd.read_json('reviewSelected100.json', lines=True, encoding = "ISO-8859-1")

In [3]:
df.describe()

,stars,useful,funny,cool
count,20.000000,20.00000,20.000000,20.000000
mean,3.400000,1.90000,0.650000,0.750000
std,1.698296,2.35975,1.424411,1.773341
min,1.000000,0.00000,0.000000,0.000000
25%,1.750000,0.00000,0.000000,0.000000
50%,4.000000,1.00000,0.000000,0.000000
75%,5.000000,4.00000,0.250000,0.000000
max,5.000000,7.00000,5.000000,6.000000


In [4]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59
1,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3,5,4,5,Tracy dessert had a big name in Hong Kong and ...,2016-05-07 01:21:02
2,rEITo90tpyKmEfNDp3Ou3A,6Fz_nus_OG4gar721OKgZA,6lj2BJ4tJeu7db5asGHQ4w,5,0,0,0,We've been a huge Slim's fan since they opened...,2017-05-26 01:23:19
3,bjD0Dqn3k-fi00BXatrytg,1fi6x4tnJtlVWaJmoIO9XA,XTzKRvWciP_BZ9imk8mmPQ,1,5,1,0,I tried this place because my girls are away f...,2014-06-27 21:32:31
4,CelUWzp-GnJIiiV1mDUb-g,tFICmdLtwgFIRcwtlbYQOg,CDpoJiqgM04wqQTZ6QkTqQ,1,0,0,0,Love this place downtown but the Scottsdale lo...,2015-12-05 02:37:03


Most frequent ⟨ Noun - Adjective ⟩ pairs for each rating. Each review has a “star” rating in the range of
1 to 5. Randomly select 50 reviews (one from each business) of rating 1, extract the top-10 most frequent
noun-adjective pairs from the sentences in these selected reviews. Example noun-adjective pairs are servicegreat,
food-delicious, that appear in the same sentence. Do the same for 20 reviews of ratings 2, 3, 4, and 5,
respectively. Discuss your results and limitations of your method.

In [97]:
# random 50 reviews (not the same biz) from rating 1

# 2 ways to do it
# 1. get random biz, then from there get one random review 
# 2. just get 50 random review, then if there is any other biz that overlap then find random again, repeat until all
#    review is from different biz (this)


def get_all_biz(df):
    biz = df['business_id'].unique()
    sorted(biz)
    return biz

def get_random(df_len, seed=10086):
    return random.randint(0,df_len)    

def extract_random_reviews_from_different_biz(num_reviews, star_rating, df):
    # filter star rating
    filter_df = df.loc[df['stars'] == star_rating]
    
    # get random review
    random_review = {}
    while (len(random_review.keys()) < num_reviews):
        random_index = get_random(len(filter_df)-1)
        biz_name = filter_df.iloc[[random_index]]['business_id'].values[0]
        if biz_name in random_review:
            continue
        else:
            random_review[biz_name] = filter_df.iloc[[random_index]]['text'].values[0]
            #filter_df = filter_df.drop([filter_df.index[random_index]])
    
    
    # convert to list
    return list(random_review.values())

In [88]:
nlp = spacy.load('en_core_web_sm')

In [100]:
# Visualise dependencies
displacy.render(nlp("Met a friend for dinner there tonight. The server only brought us one menu. After she brought us a couple of Cokes she didn't come to our table for 20 mins. After trying to get her attention a few times when she walked by we decided to just order off the bartender. We received the starters and the mains at the same time. The wings, brisket, and mac and cheese were alright at best. At this point we hadn't seen the server since she brought the food, so we had to go to the bar for more napkins and refills on the Cokes. The most we actually saw our server was when she hovered over us as we put the tip into the handheld credit card terminal. \n\nFood was somewhere between okay and decent. Service was somewhere between piss poor and terrible."), style='dep', jupyter=True)

In [93]:
# Visualise dependencies
displacy.render(nlp('The sandwich was excellent, served on perfectly crisp, chewy saltimbocca bread.'), style='dep', jupyter=True)

In [ ]:
# do i need to clean data (eg. \n\n) / any preprocessing
# change everything to singular? but this one requires stemming, then stemming might be off too

In [108]:
# def get_noun_adj_pair(reviews):
#     noun_adj_dict = {}
#     for k in range(len(reviews)):
#         noun_adj_pairs = []
#         doc = nlp(reviews[k])
#         noun_adj_pairs = []
#         for np in doc.noun_chunks: # np refers to noun phrases
#             np_adj = []
#             np_noun = []
#             sub_np_adj = []
#             for token in np: # handles cases where adjectives are right before the noun e.g. "great food"
#                 if token.pos_ in ("ADV","ADJ","PART"):
#                     sub_np_adj.append(token)
#                 elif (token.pos_ in ("NOUN", "PROPN") and str(token) != 'one'):
#                       np_noun.append(token)
#             if len(sub_np_adj)>0:
#                 np_adj.append(sub_np_adj)
#             # handles sentences with auxiliary such as "food was great"
#             sub_np_adj = []
#             for j in range(token.i+1,len(doc)): # scan rest of review for an auxiliary such as "was, were, is, are"
#                 if doc[j].pos_ in ('AUX'):
#                     aux_found = False
#                     neg_conj_found = False
#                     for child in list(doc[j].children): # search through the children of the auxiliary
#                         if neg_conj_found == False:
#                             sub_np_adj = []
#                         else:
#                             neg_conj_found = False
#                         if child.pos_ in ("PART","CCONJ"):
#                             neg_conj_found = True
#                         if (child.pos_ in ("NOUN","PROPN") and str(child) != 'one') and (child in np):
#                             aux_found = True # start looking for corresponding adjective when noun is found
#                         if aux_found == True and child.pos_ in ("ADJ","ADV","PART"): # PART refers to negation
#                             for grandchild in child.children: # extracts the "very" in "very romantic outside"
#                                 if grandchild.pos_ in ("ADJ","ADV","PART") and grandchild.i<child.i:
#                                     sub_np_adj.append(grandchild)
#                             sub_np_adj.append(child)
#                             if sub_np_adj not in np_adj:
#                                 np_adj.append(sub_np_adj)

#             if (len(np_noun)!=0 and len(np_adj)!=0):
#                 for sub_np_adj in np_adj:
#                     noun_adj_pairs.append((" ".join([str(x).lower() for x in np_noun])," ".join([str(x).lower() for x in sub_np_adj])))

#         noun_adj_dict[k+1] = noun_adj_pairs
#     return noun_adj_dict

In [116]:
def get_noun_adj_pair(reviews):
    noun_adj_dict = {}
    for k in range(len(reviews)):
        doc = nlp(reviews[k])
        noun_adj_pairs = []
        for np in doc.noun_chunks: # np refers to noun phrases
            np_adj = []
            np_noun = []
            sub_np_adj = []
            for token in np: # handles cases where adjectives are right before the noun e.g. "great food"
                if token.pos_ in ("ADV","ADJ","PART"):
                    sub_np_adj.append(token)
                elif (token.pos_ in ("NOUN", "PROPN") and str(token) != 'one'):
                      np_noun.append(token)
            if len(sub_np_adj)>0:
                np_adj.append(sub_np_adj)
            # handles sentences with auxiliary such as "food was great"
            sub_np_adj = []
            for j in range(token.i+1,len(doc)): # scan rest of review for an auxiliary such as "was, were, is, are"
                if doc[j].pos_ in ('AUX'):
                    aux_found = False
                    neg_conj_found = False
                    for child in list(doc[j].children): # search through the children of the auxiliary
                        if neg_conj_found == False:
                            sub_np_adj = []
                        else:
                            neg_conj_found = False
                        if child.pos_ in ("PART","CCONJ"):
                            neg_conj_found = True
                        if (child.pos_ in ("NOUN","PROPN") and str(child) != 'one') and (child in np):
                            aux_found = True # start looking for corresponding adjective when noun is found
                        if aux_found == True and child.pos_ in ("ADJ","ADV","PART"): # PART refers to negation
                            for grandchild in child.children: # extracts the "very" in "very romantic outside"
                                if grandchild.pos_ in ("ADJ","ADV","PART") and grandchild.i<child.i:
                                    sub_np_adj.append(grandchild)
                            sub_np_adj.append(child)
                            if sub_np_adj not in np_adj:
                                np_adj.append(sub_np_adj)

            if (len(np_noun)!=0 and len(np_adj)!=0):
                for sub_np_adj in np_adj:
                    #noun_adj_pairs.append((" ".join([str(x).lower() for x in np_noun])," ".join([str(x).lower() for x in sub_np_adj])))
                    noun_adj_pair = (" ".join([str(x).lower() for x in np_noun])," ".join([str(x).lower() for x in sub_np_adj]))
                    if noun_adj_pair in noun_adj_dict:
                        noun_adj_dict[noun_adj_pair] += 1
                    else:
                        noun_adj_dict[noun_adj_pair] = 1
                        
    return noun_adj_dict

## Actual data

In [113]:
df2 = pd.read_json('reviewSelected100.json', lines = True, encoding = "ISO-8859-1")

In [117]:
noun_adj_pairs = get_noun_adj_pair(extract_random_reviews_from_different_biz(20, 1, df2))

In [118]:
print(noun_adj_pairs)

{('soup', 'really really bad'): 1, ('excuse', 'sorry'): 1, ('restaurant', 'sushi'): 1, ('hotel', 'beautiful'): 1, ('location', 'wonderful'): 1, ('room', 'dirty'): 1, ('level', 'respectable'): 1, ('start', 'bad'): 1, ('time', 'good'): 2, ('drinks', 'just few'): 1, ('thing', 'little'): 2, ('day', 'next'): 1, ('bars', 'not'): 1, ('hour', 'first'): 1, ('group', 'small'): 1, ('year women', 'old'): 1, ('party', 'wildest'): 1, ('place', 'horrible'): 1, ('rent', 'astronomical'): 1, ('complex', 'constantly dirty'): 1, ('joke', 'absolute'): 1, ('account statement', 'final'): 1, ('bull', 'total'): 1, ('blinds', 'broken'): 1, ('news channel', 'well respected'): 1, ('franchise', 'whole'): 1, ('bar food', 'typical'): 1, ('locations', 'other'): 1, ('meals events', 'special'): 1, ('digs', 'new'): 1, ('communication efforts', 'multiple'): 1, ('flag', 'red'): 1, ('star', 'negative'): 1, ('price', 'different'): 1, ('girls', 'so rude'): 1, ('poutine', 'gross'): 1, ('brewsters poutine', 'better'): 1, ('bee

In [ ]:
# sort and get top 5
# Coreference Resolution?

for testing

In [102]:
reviews = extract_random_reviews_from_different_biz(2, 1, df)

In [107]:
print(get_noun_adj_pair(reviews))

{('time', 'same'): 1, ('wings', 'alright'): 1, ('wings', 'at best'): 1, ('napkins', 'more'): 1, ('credit card terminal', 'handheld'): 1, ('food', 'somewhere'): 1, ('service', 'somewhere'): 1, ('fees', 'extra'): 1, ('toenail', 'loose'): 1, ('stone massage', "'"): 1, ('stone massage', 'barely'): 1, ('legs', 'lower'): 1, ('lines', 'few'): 1, ('dollars', 'more'): 1, ('place', 'appalling'): 1}


In [109]:
print(get_noun_adj_pair(reviews))

{1: [('time', 'same'), ('wings', 'alright'), ('wings', 'at best'), ('napkins', 'more'), ('credit card terminal', 'handheld'), ('food', 'somewhere'), ('service', 'somewhere')], 2: [('fees', 'extra'), ('toenail', 'loose'), ('stone massage', "'"), ('stone massage', 'barely'), ('legs', 'lower'), ('lines', 'few'), ('dollars', 'more'), ('place', 'appalling')]}


In [105]:
print(get_noun_adj_pair1(reviews))

{('', 'more'): 1, ('', 'extra'): 1}


In [80]:
get_all_biz(df)

array(['eU_713ec6fTGNO4BegRaww', '3fw2X5bZYeW9xCz_zGhOHg',
       '6lj2BJ4tJeu7db5asGHQ4w', 'XTzKRvWciP_BZ9imk8mmPQ',
       'CDpoJiqgM04wqQTZ6QkTqQ', 'm7ommfJFalI47UksyX04Iw',
       '6sATfFam9_q9uod0I2aCsg', 'SU56w479vUfFHsvmvQIf7A',
       'CGUK3cd2gxp2q3KPY19Oog', 'vhIJ91MDgUuk4Cr9Kpj1Nw',
       '8KmqWgL0UEdxJFwTZ_YZvQ', 'EgwGTDZ705TwudPJwAY0yQ',
       'GdCRQU3VCh_x8fY84mbwYQ', 'qaPSbg690KaX5av6xsSV4Q',
       '9mIm1ef-NVDQHFE39Faxxg', '9nHpqlu7be1466wdo_t5kQ',
       'FStFa2esHFgsAjVzIGJcQA', 'Z66xO_B7trDah8F0PKwRqg',
       'Nc7cqq5k0WZsVhu25QrmNQ'], dtype=object)

In [14]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59
1,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3,5,4,5,Tracy dessert had a big name in Hong Kong and ...,2016-05-07 01:21:02
2,rEITo90tpyKmEfNDp3Ou3A,6Fz_nus_OG4gar721OKgZA,6lj2BJ4tJeu7db5asGHQ4w,5,0,0,0,We've been a huge Slim's fan since they opened...,2017-05-26 01:23:19
3,bjD0Dqn3k-fi00BXatrytg,1fi6x4tnJtlVWaJmoIO9XA,XTzKRvWciP_BZ9imk8mmPQ,1,5,1,0,I tried this place because my girls are away f...,2014-06-27 21:32:31
4,CelUWzp-GnJIiiV1mDUb-g,tFICmdLtwgFIRcwtlbYQOg,CDpoJiqgM04wqQTZ6QkTqQ,1,0,0,0,Love this place downtown but the Scottsdale lo...,2015-12-05 02:37:03


In [46]:
biz = df.iloc[[2]]
print(biz['business_id'].values[0])

6lj2BJ4tJeu7db5asGHQ4w


In [29]:
type(df.iloc[[2]]['business_id'])

pandas.core.series.Series